In [1]:
import pandas as pd
import numpy as np
import shap
import joblib
import json
import warnings
from collections import defaultdict

warnings.filterwarnings('ignore')
print('✅ Dependencies loaded.')

✅ Dependencies loaded.


In [2]:
# Load model artifacts via joblib
pd_model = joblib.load('pd_model.joblib')
pd_scaler = joblib.load('pd_scaler.joblib')
pd_features = joblib.load('pd_features.joblib')

iso_model = joblib.load('isolation_forest.joblib')
if_scaler = joblib.load('if_scaler.joblib')
if_features = ['avgMonthlyIncome', 'incomeCV', 'expenseRatio', 'emiRatio', 'avgMonthlyBalance', 'bounceCount']

risk_model = joblib.load('risk_random_forest.joblib')
risk_features = joblib.load('risk_features.joblib')

hybrid_model = joblib.load('hybrid_credit_score_model.joblib')
hybrid_features = joblib.load('hybrid_features.joblib')

q_table = joblib.load('q_learning_model.joblib')
q_bins = joblib.load('q_learning_bins.joblib')
q_features = joblib.load('q_learning_features.joblib')

bg_data = pd.read_csv('../data/synthetic/features_only.csv')
print('✅ Models and artifacts loaded successfully from joblib.')

✅ Models and artifacts loaded successfully from joblib.


In [3]:
def get_unified_explanation(input_row):
    df_input = pd.DataFrame([input_row])
    
    # 1. PD
    X_pd = pd_scaler.transform(df_input[pd_features])
    pd_val = pd_model.predict_proba(X_pd)[0, 1]

    # 2. Anomaly
    X_if = if_scaler.transform(df_input[if_features])
    if_score = iso_model.decision_function(X_if)[0]

    # 3. Risk Label
    df_risk = df_input.copy()
    df_risk['PD'] = pd_val
    df_risk['anomalyFlag'] = 1 if if_score < -0.05 else 0
    X_risk = df_risk[risk_features]
    risk_idx = int(risk_model.predict(X_risk)[0])
    risk_label = ["LOW", "MEDIUM", "HIGH"][risk_idx]

    # 4. Hybrid Score
    X_hyb = df_input[hybrid_features]
    score_val = hybrid_model.predict(X_hyb)[0]

    # 5. RL Action
    anom_norm = np.clip(1.0 - (if_score + 0.5), 0, 1)
    X_rl = np.array([[pd_val, anom_norm, score_val]])
    s = (np.digitize(X_rl[0][0], q_bins['pd']),
         np.digitize(X_rl[0][1], q_bins['anom']),
         np.digitize(X_rl[0][2], q_bins['cs']))
    
    q_vals = q_table.get(s, np.zeros(4))
    action_idx = int(np.argmax(q_vals))
    
    actions = ["REJECT", "APPROVE_LOW", "APPROVE_MEDIUM", "APPROVE_HIGH"]
    

    return {
        "PD": {"value": round(float(pd_val), 4)},
        "Anomaly": {"score": round(float(if_score), 4)},
        "RiskLabel": {"label": risk_label},
        "HybridScore": {"value": round(float(score_val), 1)},
        "RL_Recommendation": {"action": actions[action_idx]}
    }

print('✅ Unified Aggregator defined.')

✅ Unified Aggregator defined.


In [4]:
test_customer1 = {
    'avgMonthlyIncome': 315210.9, 'incomeCV': 0.05, 'expenseRatio': 0.314, 
    'emiRatio': 0.121, 'avgMonthlyBalance': 67185.96, 'bounceCount': 0, 'accountAgeMonths': 100
}
print('--- Unified Explanation Response ---')
print(json.dumps(get_unified_explanation(test_customer1), indent=2))

--- Unified Explanation Response ---
{
  "PD": {
    "value": 0.0034
  },
  "Anomaly": {
    "score": 0.1466
  },
  "RiskLabel": {
    "label": "LOW"
  },
  "HybridScore": {
    "value": 668.9
  },
  "RL_Recommendation": {
    "action": "APPROVE_HIGH"
  }
}


In [5]:
test_customer2 = {
    'avgMonthlyIncome': 319580.3, 'incomeCV': 0.537, 'expenseRatio': 0.916, 
    'emiRatio': 0.371, 'avgMonthlyBalance': 117421.71, 'bounceCount': 1, 'accountAgeMonths': 26
}
print('--- Unified Explanation Response ---')
print(json.dumps(get_unified_explanation(test_customer2), indent=2))

--- Unified Explanation Response ---
{
  "PD": {
    "value": 0.8925
  },
  "Anomaly": {
    "score": 0.1446
  },
  "RiskLabel": {
    "label": "HIGH"
  },
  "HybridScore": {
    "value": 420.0
  },
  "RL_Recommendation": {
    "action": "REJECT"
  }
}
